In [4]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [371]:
class person:
    def __init__(self,i,gender, orientation, age, sexual_activity):
        # gender 0 : male, 1 : female
        # orientation 0 : straight, 1 : homosexual
        # time_since_infection equals -1 if not infected
        
        self.identifier = i
        self.gender = gender
        self.orientation = orientation
        self.age = age
        self.sexual_activity = sexual_activity
        self.disease_status = 0
        self.time_since_infection = -1
        self.number_of_partners = 0
        self.current_partners = set()
    
    def print_state(self):
        print(self.identifier, 
              self.gender, 
              self.orientation,
              self.age,
              self.sexual_activity,
              self.time_since_infection,
              self.number_of_partners,
              self.current_partners)
    
    def add_partner(self, other_person):
        self.number_of_partners += 1
        self.current_partners.add(other_person.identifier)
        
    def remove_partner(self, other_person):
        self.number_of_partners -= 1
        self.current_partners.remove(other_person.identifier)

class Partnership:
    def __init__(self,partnership_type,person1,person2):
        """
        type 1 : steady, 2 : casual
        
        """
        self.type = partnership_type,
        self.persons = [person1,person2]
        
class system:
    def __init__(self):
        self.persons = []
        self.partnerships = []
        self.straight_males = []
        self.homosexual_males = []
        self.females = []
    
    def set_constants(self,
                      f = 0.2,
                      rho = 0.006,
                      sigma1 = 0.0004,
                      sigma2 = 0.1,
                      transmission_male_female_steady = 0.15,
                      transmission_female_male_steady = 0.0625,
                      transmission_male_female_casual = 0.6,
                      transmission_female_male_casual = 0.25,
                      ratio_asymptomatic_men = 0.1,
                      ratio_asymptomatic_women = 0.45,
                      incubation_time_men = 5,
                      incubation_time_women = 10,
                      patient_delay_treatment_men = 5,
                      patient_delay_treatment_women = 8,
                      recovery_rate_asymptomatic_men = 0.0074,
                      recovery_rate_symptomatic_men = 0.04,
                      recovery_rate_asymptomatic_women = 0.0044,
                      recovery_rate_symptomatic_women = 0.03):
        """
        sets constants

        Definitions
        f: probability of steady partnership
        rho: probability of forming partnership
        sigma1 : probability of steady partnership separation
        sigma2 : probability of casual partnership separation
        """
        self.f = f
        self.rho = rho
        self.sigma1 = sigma1
        self.sigma2 = sigma2

        self.transmission_male_female_steady = transmission_male_female_steady
        self.transmission_female_male_steady = transmission_female_male_steady
        self.transmission_male_female_casual = transmission_male_female_casual
        self.transmission_female_male_casual = transmission_female_male_casual
        self.ratio_asymptomatic_men = ratio_asymptomatic_men
        self.ratio_asymptomatic_women = ratio_asymptomatic_women
        self.incubation_time_men = incubation_time_men
        self.incubation_time_women = incubation_time_women
        self.patient_delay_treatment_men = patient_delay_treatment_men
        self.patient_delay_treatment_women = patient_delay_treatment_women
        self.recovery_rate_asymptomatic_men = recovery_rate_asymptomatic_men
        self.recovery_rate_symptomatic_men = recovery_rate_symptomatic_men
        self.recovery_rate_asymptomatic_women = recovery_rate_asymptomatic_women
        self.recovery_rate_symptomatic_men = recovery_rate_symptomatic_men

    
    def initialize(self,number_of_persons, gender_ratio, queer_ratio):
        for i in range(number_of_persons):
            gender =  0 if np.random.random() > gender_ratio else 1
            orientation = 1 if gender == 0 and np.random.random() < queer_ratio else 0
            age = np.random.randint(15,65)
            if age >= 15 and age < 35:
                sexual_activity = 1 if np.random.random() < 0.05 else 0
            else:
                sexual_activity = 0
            
            
            p = person(i,gender, orientation, age, sexual_activity)
            self.persons.append(p)
            if gender == 0 and orientation == 0:
                self.straight_males.append(p)
            elif gender == 0 and orientation == 1:
                self.homosexual_males.append(p)
            else:
                self.females.append(p)
    
    def replace_person(self,p):
        identifier = p.identifier
        gender = p.gender
        orientation = p.orientation
        age = 15
        sexual_activity = 1 if np.random.random() < 0.05 else 0
        
        for j in p.current_partners:
            self.persons[j].remove_partner(p)
        
        new_person = person(identifier,gender, orientation, age, sexual_activity)
        
        self.persons[identifier] = new_person
    
    def print_state(self):
        print("id\tgender\torientation\tage\tactivity\tdisease\ttime since inf\t#partners\tcurrent partners")
        for p in self.persons:
            p.print_state()
            
    def time_step(self):
        n = int(len(self.persons)/2) - len(self.partnerships)
        
        #creation of partnerships
        for i in range(n):
            if np.random.random() < self.rho:
                formed = False
                partnership_type = 1 if np.random.random() < self.f else 2
                
                #add a end condition
                iteration = 0
                while(not formed and iteration < 100):
                    #form partnership
                    y = np.random.choice(self.straight_males)
                    x = np.random.choice(self.females)
                    
                    mixing_prob = mixing_probability(x,y,partnership_type)
                    if np.random.random() < mixing_prob:
                        p = Partnership(partnership_type,x,y)
                        x.add_partner(y)
                        y.add_partner(x)
                        self.partnerships.append(p)
                        formed = True
                    iteration += 1
        
        #TODO: Disease transmission
        for i,partnership in enumerate(self.partnerships):
            p1, p2 = partnership.persons
            s1 = p1.disease_status
            s2 = p2.disease_status
            partnership_type = partnership.type
            
            
#             if s1 > 0 and s2 == 0:
#                 if s1.gender == 0 and s2.gender == 1:
                    
#             elif s1 == 0 and s2 > 0:
        
        #Separation of partnerships
        del_list = []
        for i,ps in enumerate(self.partnerships):
            partnership_type = ps.type
            p1, p2 = ps.persons
            
            #TODO: check type
            if partnership_type == 1:
                if np.random.random() < self.sigma1:
                    del_list.append(i)
                    p1.remove_partner(p2)
                    p2.remove_partner(p1)
            elif partnership_type == 2:
                if np.random.random() < self.sigma2:
                    del_list.append(i)
                    p1.remove_partner(p2)
                    p2.remove_partner(p1)
        for i in reversed(sorted(del_list)):
            del(self.partnerships[i])
        
        #TODO: Replacement
        for p in self.persons:
            if p.age >= 64:
                #replace with new person
                self.replace_person(p)
        
        
        #TODO: Recovery
        
        #TODO: Treatment
        
        #TODO: Screening
        
        #TODO: Increase age
        for i,p in enumerate(self.persons):
            p.age += 1
        
def mixing_probability(x,y,partnership_type):
    j = age_group(x)
    k = age_group(y)
    
    if j == k:
        phi_a = 1
    else:
        phi_a = 0.2**(np.abs(j + 1 - k))
    
    # c = sexual_activity
    # d = number_of_partners
    
    if partnership_type == 1:
        if x.number_of_partners == 0 and y.number_of_partners == 0:
            phi_cd = 1
        else:
            phi_cd = 0
    else:
        if x.sexual_activity == 1 and y.sexual_activity == 1:
            phi_cd = 1
        elif x.sexual_activity == 1 and y.sexual_activity == 0\
        and y.number_of_partners == 0:
            phi_cd = 0.1
        elif y.sexual_activity == 1 and x.sexual_activity == 0\
        and x.number_of_partners == 0:
            phi_cd = 0.1
        elif x.sexual_activity == 0 and x.number_of_partners == 0\
        and y.sexual_activity == 0 and y.number_of_partners == 0:
            phi_cd = 0.01
        else:
            phi_cd = 0
    return phi_a * phi_cd
        
    
    
def age_group(p):
    a = p.age
    if a >= 15 and a < 25:
        age_group = 1
    elif a >= 25 and a < 35:
        age_group = 2
    elif a >= 35 and a < 45:
        age_group = 3
    elif a >= 45 and a < 55:
        age_group = 4
    else:
        age_group = 5
    return age_group
        
        
        
        
        

            

In [372]:
s = system()
s.initialize(100,0.5,0)
s.set_constants()


In [373]:
s.print_state()

id	gender	orientation	age	activity	disease	time since inf	#partners	current partners
0 1 0 52 0 -1 0 set()
1 0 0 58 0 -1 0 set()
2 1 0 25 0 -1 0 set()
3 1 0 49 0 -1 0 set()
4 1 0 49 0 -1 0 set()
5 0 0 22 0 -1 0 set()
6 1 0 20 0 -1 0 set()
7 0 0 30 0 -1 0 set()
8 1 0 54 0 -1 0 set()
9 0 0 58 0 -1 0 set()
10 0 0 35 0 -1 0 set()
11 0 0 33 0 -1 0 set()
12 0 0 35 0 -1 0 set()
13 0 0 23 0 -1 0 set()
14 0 0 23 0 -1 0 set()
15 0 0 15 0 -1 0 set()
16 0 0 16 0 -1 0 set()
17 1 0 22 0 -1 0 set()
18 1 0 43 0 -1 0 set()
19 1 0 63 0 -1 0 set()
20 1 0 15 0 -1 0 set()
21 0 0 51 0 -1 0 set()
22 1 0 45 0 -1 0 set()
23 1 0 37 0 -1 0 set()
24 0 0 23 0 -1 0 set()
25 1 0 43 0 -1 0 set()
26 0 0 20 0 -1 0 set()
27 1 0 20 0 -1 0 set()
28 1 0 44 0 -1 0 set()
29 1 0 57 0 -1 0 set()
30 0 0 21 0 -1 0 set()
31 0 0 38 0 -1 0 set()
32 0 0 58 0 -1 0 set()
33 1 0 53 0 -1 0 set()
34 1 0 56 0 -1 0 set()
35 1 0 60 0 -1 0 set()
36 1 0 24 0 -1 0 set()
37 1 0 19 0 -1 0 set()
38 1 0 27 1 -1 0 set()
39 0 0 42 0 -1 0 set()
40 0 

In [374]:
for i in range(100):
    s.time_step()

KeyError: 67

In [375]:
s.print_state()

id	gender	orientation	age	activity	disease	time since inf	#partners	current partners
0 1 0 29 0 -1 0 set()
1 0 0 35 0 -1 0 set()
2 1 0 51 0 -1 0 set()
3 1 0 26 0 -1 0 set()
4 1 0 26 0 -1 0 set()
5 0 0 48 0 -1 0 set()
6 1 0 46 0 -1 1 {77}
7 0 0 56 0 -1 0 set()
8 1 0 31 0 -1 0 set()
9 0 0 35 0 -1 0 set()
10 0 0 61 0 -1 0 set()
11 0 0 59 0 -1 1 {37}
12 0 0 61 0 -1 0 set()
13 0 0 49 0 -1 0 set()
14 0 0 49 0 -1 0 set()
15 0 0 41 0 -1 0 set()
16 0 0 42 0 -1 0 set()
17 1 0 48 0 -1 0 set()
18 1 0 20 0 -1 0 set()
19 1 0 40 0 -1 0 set()
20 1 0 41 0 -1 0 set()
21 0 0 28 0 -1 0 set()
22 1 0 22 0 -1 0 set()
23 1 0 63 0 -1 0 set()
24 0 0 49 0 -1 0 set()
25 1 0 20 0 -1 0 set()
26 0 0 46 0 -1 0 set()
27 1 0 46 0 -1 1 {77}
28 1 0 21 0 -1 0 set()
29 1 0 34 0 -1 0 set()
30 0 0 47 0 -1 0 set()
31 0 0 15 0 -1 0 set()
32 0 0 35 0 -1 0 set()
33 1 0 30 0 -1 0 set()
34 1 0 33 0 -1 0 set()
35 1 0 37 0 -1 0 set()
36 1 0 50 0 -1 0 set()
37 1 0 45 0 -1 1 {11}
38 1 0 53 1 -1 1 {46}
39 0 0 19 0 -1 0 set()
40 0 0 31 